# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [1]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import matplotlib.pyplot as plt
import pyvista
from dolfinx import nls
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot
L = 20.0
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron)
V = fem.VectorFunctionSpace(domain, ("Lagrange", 2))

We create two python functions for determining the facets to apply boundary conditions to

In [2]:
def left(x):
    return np.isclose(x[0], 0)


def right(x):
    return np.isclose(x[0], L)


fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

Next, we create a  marker based on these two functions

In [3]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

We then create a function for supplying the boundary condition on the left side, which is fixed.

In [4]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [5]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`).

In [6]:
B = fem.Constant(domain, default_scalar_type((0, 0, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

Define the test and solution functions on the space $V$

In [7]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [8]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [9]:
# Elasticity parameters
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [10]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [11]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [12]:
problem = NonlinearProblem(F, u, bcs)

and then create and customize the Newton solver

In [13]:
solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"


We create a function to plot the solution at each time step.

In [14]:
pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("deformation.gif", fps=3)

topology, cells, geometry = plot.create_vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.FunctionSpace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

Finally, we solve the problem over several time steps, updating the y-component of the traction

In [15]:
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert (converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    plotter.update_coordinates(warped_n.points.copy(), render=False)
    plotter.update_scalar_bar_range([0, 10])
    plotter.update_scalars(magnitude.x.array)
    plotter.write_frame()
plotter.close()

2023-07-05 12:49:44.484 (   5.121s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:49:44.782 (   5.419s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.297925, 0.240000, 0.050000 (PETSc Krylov solver)
2023-07-05 12:49:44.893 (   5.529s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:49:45.123 (   5.759s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.230006, 0.210000, 0.030000 (PETSc Krylov solver)
2023-07-05 12:49:45.129 (   5.765s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 22.2455 (tol = 1e-08) r (rel) = 0.134278(tol = 1e-08)
2023-07-05 12:49:45.234 (   5.870s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:49:45.460 (   6.096s) [main            ]         TimeLogger.cpp:

Time step 1, Number of iterations 8, Load [ 0.   0.  -1.5]


2023-07-05 12:49:47.477 (   8.113s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:49:47.708 (   8.345s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.231524, 0.210000, 0.030000 (PETSc Krylov solver)
2023-07-05 12:49:47.822 (   8.458s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:49:48.050 (   8.687s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.228604, 0.210000, 0.020000 (PETSc Krylov solver)
2023-07-05 12:49:48.055 (   8.692s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 17.3254 (tol = 1e-08) r (rel) = 0.117842(tol = 1e-08)
2023-07-05 12:49:48.164 (   8.801s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:49:48.393 (   9.029s) [main            ]         TimeLogger.cpp:

Time step 2, Number of iterations 9, Load [ 0.  0. -3.]


2023-07-05 12:49:50.862 (  11.498s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.233915, 0.210000, 0.030000 (PETSc Krylov solver)
2023-07-05 12:49:50.984 (  11.621s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:49:51.226 (  11.862s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.241826, 0.210000, 0.020000 (PETSc Krylov solver)
2023-07-05 12:49:51.232 (  11.868s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 10.0011 (tol = 1e-08) r (rel) = 0.0887471(tol = 1e-08)
2023-07-05 12:49:51.343 (  11.979s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:49:51.597 (  12.233s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.253656, 0.230000, 0.020000 (PETSc Krylov solver)
2023-07-05 12:49:51.603 (  12.240s) [main  

...

Time step 7, Number of iterations 6, Load [  0.    0.  -10.5]


2023-07-05 12:50:05.212 (  25.848s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:50:05.451 (  26.087s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.238825, 0.220000, 0.020000 (PETSc Krylov solver)
2023-07-05 12:50:05.567 (  26.203s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:50:05.806 (  26.442s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.239502, 0.210000, 0.040000 (PETSc Krylov solver)
2023-07-05 12:50:05.812 (  26.449s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 1.06336 (tol = 1e-08) r (rel) = 0.031085(tol = 1e-08)
2023-07-05 12:50:05.926 (  26.562s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:50:06.161 (  26.797s) [main            ]         TimeLogger.cpp:

Time step 8, Number of iterations 6, Load [  0.   0. -12.]


2023-07-05 12:50:07.245 (  27.882s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 6: r (abs) = 3.24402e-11 (tol = 1e-08) r (rel) = 9.48314e-13(tol = 1e-08)
2023-07-05 12:50:07.245 (  27.882s) [main            ]       NewtonSolver.cpp:255   INFO| Newton solver finished in 6 iterations and 6 linear solver iterations.
2023-07-05 12:50:07.382 (  28.019s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:50:07.619 (  28.255s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.236171, 0.200000, 0.040000 (PETSc Krylov solver)
2023-07-05 12:50:07.737 (  28.373s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-07-05 12:50:07.972 (  28.609s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.235308, 0.200000, 0.030000 (PETSc Krylov solver)
2023-07-05 12:50:07.978 (  28.614s) [main     

Time step 9, Number of iterations 6, Load [  0.    0.  -13.5]


<img src="./deformation.gif" alt="gif" class="bg-primary mb-1" width="800px">